In [11]:
import numpy as np
#import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import autograd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
transform_test = transforms.Compose([
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
batch_size = 128
testset = torchvision.datasets.CIFAR10(root='./', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)
testloader = enumerate(testloader)
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3  ,196, 3,1,1)
        self.conv2 = nn.Conv2d(196,196, 3,2,1)
        self.conv3 = nn.Conv2d(196,196, 3,1,1)
        self.conv4 = nn.Conv2d(196,196, 3,2,1)
        self.conv5 = nn.Conv2d(196,196, 3,1,1)
        self.conv6 = nn.Conv2d(196,196, 3,1,1)
        self.conv7 = nn.Conv2d(196,196, 3,1,1)
        self.conv8 = nn.Conv2d(196,196, 3,2,1)
        
        self.ln1 = nn.LayerNorm((196,32,32))
        self.ln2 = nn.LayerNorm((196,16,16))
        self.ln3 = nn.LayerNorm((196,16,16))
        self.ln4 = nn.LayerNorm((196,8,8))
        self.ln5 = nn.LayerNorm((196,8,8))
        self.ln6 = nn.LayerNorm((196,8,8))
        self.ln7 = nn.LayerNorm((196,8,8))
        self.ln8 = nn.LayerNorm((196,4,4))
        
        
        
        self.pool = nn.MaxPool2d(4,4)           # 4*4
        self.lkrl = nn.LeakyReLU()
        self.fc1 = nn.Linear(196, 1)            # 1     critic output
        self.fc10 = nn.Linear(196, 10)          # 10    auxiliary 
        
    def forward(self, x):
        #layer1
        x = self.conv1(x)
        x = self.ln1(x)
        x = F.leaky_relu(x)
        
        x = self.conv2(x)
        x = self.ln2(x)
        x = F.leaky_relu(x)
       
        x = self.conv3(x)
        x = self.ln3(x)
        x = F.leaky_relu(x)
        
        x = self.conv4(x)
        x = self.ln4(x)
        x = F.leaky_relu(x)
        
        x = self.conv5(x)
        x = self.ln5(x)
        x = F.leaky_relu(x)
        
        x = self.conv6(x)
        x = self.ln6(x)
        x = F.leaky_relu(x)
        
        x = self.conv7(x)
        x = self.ln7(x)
        x = F.leaky_relu(x)
        
        x = self.conv8(x)
        x = self.ln8(x)
        x = F.leaky_relu(x)
        
        x = self.pool(x)
        x1 = x.view(-1, 196)
        critic = self.fc1(x1)
        x2 = x.view(-1, 196)
        classifier = self.fc10(x2)
        return critic,classifier

model = torch.load('cifar10.model')
model.to(device)
model.eval()

C:\Users\wangwang\Anaconda3\lib\site-packages\torch\serialization.py:391: UserWarning: Couldn't retrieve source code for container of type discriminator. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


discriminator(
  (conv1): Conv2d(3, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv5): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(196, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(196, 196, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (ln1): LayerNorm(torch.Size([196, 32, 32]), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
  (ln3): LayerNorm(torch.Size([196, 16, 16]), eps=1e-05, elementwise_affine=True)
  (ln4): LayerNorm(torch.Size([196, 8, 8]), eps=1e-05, elementwise_affine=True)
  (ln5): LayerNorm(torch.Size

In [13]:
batch_idx, (X_batch, Y_batch) = testloader.__next__()
X_batch = Variable(X_batch,requires_grad=True).cuda()
Y_batch_alternate = (Y_batch + 1)%10
Y_batch_alternate = Variable(Y_batch_alternate).cuda()
Y_batch = Variable(Y_batch).cuda()

In [14]:
def plot(samples):
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(10, 10)
    gs.update(wspace=0.02, hspace=0.02)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample)
    return fig
## save real images
samples = X_batch.data.cpu().numpy()
samples += 1.0
samples /= 2.0
samples = samples.transpose(0,2,3,1)

fig = plot(samples[0:100])
plt.savefig('visualization/real_images.png', bbox_inches='tight')
plt.close(fig)

In [15]:
_, output = model(X_batch)
prediction = output.data.max(1)[1] # first column has actual prob.
accuracy = ( float( prediction.eq(Y_batch.data).sum() ) /float(batch_size))*100.0
print(accuracy)


93.75


In [16]:
## slightly jitter all input images
criterion = nn.CrossEntropyLoss(reduce=False)
loss = criterion(output, Y_batch_alternate)

gradients = torch.autograd.grad(outputs=loss, inputs=X_batch,
                          grad_outputs=torch.ones(loss.size()).cuda(),
                          create_graph=True, retain_graph=False, only_inputs=True)[0]

# save gradient jitter
gradient_image = gradients.data.cpu().numpy()
gradient_image = (gradient_image - np.min(gradient_image))/(np.max(gradient_image)-np.min(gradient_image))
gradient_image = gradient_image.transpose(0,2,3,1)
fig = plot(gradient_image[0:100])
plt.savefig('visualization/gradient_image.png', bbox_inches='tight')
plt.close(fig)

C:\Users\wangwang\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [18]:
# jitter input image
gradients[gradients>0.0] = 1.0
gradients[gradients<0.0] = -1.0

gain = 8.0
X_batch_modified = X_batch - gain*0.007843137*gradients
X_batch_modified[X_batch_modified>1.0] = 1.0
X_batch_modified[X_batch_modified<-1.0] = -1.0

## evaluate new fake images
_, output = model(X_batch_modified)
prediction = output.data.max(1)[1] # first column has actual prob.
accuracy = ( float( prediction.eq(Y_batch.data).sum() ) /float(batch_size))*100.0
print(accuracy)

## save fake images
samples = X_batch_modified.data.cpu().numpy()
samples += 1.0
samples /= 2.0
samples = samples.transpose(0,2,3,1)

fig = plot(samples[0:100])
plt.savefig('visualization/jittered_images.png', bbox_inches='tight')
plt.close(fig)

14.84375


part3-2-without gen

In [17]:
X = X_batch.mean(dim=0)
X = X.repeat(10,1,1,1)

Y = torch.arange(10).type(torch.int64)
Y = Variable(Y).cuda()

In [19]:

lr = 0.1
weight_decay = 0.001
for i in range(200):
    _, output = model(X)

    loss = -output[torch.arange(10).type(torch.int64),torch.arange(10).type(torch.int64)]
    gradients = torch.autograd.grad(outputs=loss, inputs=X,
                              grad_outputs=torch.ones(loss.size()).cuda(),
                              create_graph=True, retain_graph=False, only_inputs=True)[0]

    prediction = output.data.max(1)[1] # first column has actual prob.
    accuracy = ( float( prediction.eq(Y.data).sum() ) /float(10.0))*100.0
    print(i,accuracy,-loss)

    X = X - lr*gradients.data - weight_decay*X.data*torch.abs(X.data)
    X[X>1.0] = 1.0
    X[X<-1.0] = -1.0

## save new images
samples = X.data.cpu().numpy()
samples += 1.0
samples /= 2.0
samples = samples.transpose(0,2,3,1)

fig = plot(samples)
plt.savefig('visualization/max_class.png', bbox_inches='tight')
plt.close(fig)

0 10.0 tensor([-1.9523, -8.3031, -5.3835, -3.3779,  3.4734, -2.5692,  0.3218, -3.7604,
         1.3529, -7.3200], device='cuda:0', grad_fn=<NegBackward>)
1 20.0 tensor([-1.2641, -6.0769, -1.1562,  0.0495, 10.4268, -2.8045,  8.9206, -2.9196,
         0.5854, -4.6089], device='cuda:0', grad_fn=<NegBackward>)
2 80.0 tensor([12.8713, -1.4993,  9.9239,  8.9318, 16.5689, 11.3143, 15.3464,  5.2597,
        10.3821,  1.5906], device='cuda:0', grad_fn=<NegBackward>)
3 100.0 tensor([15.7657,  6.9913, 17.9493, 12.6229, 20.5418, 17.0861, 18.6997, 14.9550,
        17.5291,  7.3929], device='cuda:0', grad_fn=<NegBackward>)
4 100.0 tensor([20.6962, 16.5413, 20.7197, 16.9426, 22.2182, 21.0254, 21.5612, 19.2881,
        20.8566, 15.0244], device='cuda:0', grad_fn=<NegBackward>)
5 100.0 tensor([22.8336, 20.2841, 22.6097, 19.8975, 24.7887, 23.0814, 23.7834, 21.6362,
        23.0379, 18.0481], device='cuda:0', grad_fn=<NegBackward>)
6 100.0 tensor([24.6332, 22.5273, 23.9452, 20.7104, 25.7192, 24.0115, 24.

63 100.0 tensor([30.7893, 28.2029, 31.0241, 26.5034, 33.0174, 28.6582, 29.6132, 32.3175,
        28.6927, 27.9890], device='cuda:0', grad_fn=<NegBackward>)
64 100.0 tensor([30.8355, 28.3525, 31.1988, 25.8266, 33.1825, 29.1482, 30.4683, 32.3303,
        28.9784, 27.8265], device='cuda:0', grad_fn=<NegBackward>)
65 100.0 tensor([30.7919, 28.1825, 31.1742, 26.4991, 33.0942, 29.3292, 30.4469, 32.1081,
        28.7608, 27.8520], device='cuda:0', grad_fn=<NegBackward>)
66 100.0 tensor([30.8675, 28.3220, 31.0923, 26.7909, 33.5915, 29.1846, 30.1372, 32.3620,
        29.0981, 28.3169], device='cuda:0', grad_fn=<NegBackward>)
67 100.0 tensor([31.0984, 28.3307, 31.0829, 27.1401, 33.4919, 29.1016, 30.1382, 32.0936,
        28.8940, 28.4521], device='cuda:0', grad_fn=<NegBackward>)
68 100.0 tensor([31.0213, 28.3542, 31.1219, 27.1706, 33.5412, 29.3086, 30.4782, 32.4400,
        28.9626, 28.4871], device='cuda:0', grad_fn=<NegBackward>)
69 100.0 tensor([30.9660, 28.3756, 30.9185, 27.0182, 33.4745, 29

125 100.0 tensor([32.0865, 29.6113, 32.0430, 27.5231, 34.0868, 30.1264, 30.8546, 33.4899,
        30.3083, 29.3830], device='cuda:0', grad_fn=<NegBackward>)
126 100.0 tensor([31.7467, 29.7069, 31.6770, 27.6594, 33.9849, 30.1185, 31.1304, 33.3790,
        30.3197, 29.3570], device='cuda:0', grad_fn=<NegBackward>)
127 100.0 tensor([31.8288, 29.6011, 32.0641, 27.7861, 34.1755, 30.0972, 31.3300, 33.4486,
        30.2169, 29.6548], device='cuda:0', grad_fn=<NegBackward>)
128 100.0 tensor([31.8183, 29.6523, 31.8467, 27.7783, 33.8634, 29.8014, 31.4423, 33.2583,
        29.9929, 29.5944], device='cuda:0', grad_fn=<NegBackward>)
129 100.0 tensor([32.0648, 29.2820, 31.9241, 27.6624, 34.2788, 30.2329, 31.4501, 33.4767,
        29.4282, 29.5300], device='cuda:0', grad_fn=<NegBackward>)
130 100.0 tensor([31.8954, 29.1810, 32.0091, 27.7138, 34.2706, 30.1869, 31.3614, 33.4246,
        29.9373, 29.1019], device='cuda:0', grad_fn=<NegBackward>)
131 100.0 tensor([31.9342, 28.7425, 32.4186, 27.4126, 34.3

188 100.0 tensor([32.3477, 30.2176, 32.3241, 28.5399, 34.7088, 30.2751, 31.4596, 33.8518,
        30.5025, 30.3324], device='cuda:0', grad_fn=<NegBackward>)
189 100.0 tensor([32.6108, 30.3136, 32.6638, 28.6092, 34.6143, 30.5690, 31.8084, 33.3471,
        30.4573, 30.0108], device='cuda:0', grad_fn=<NegBackward>)
190 100.0 tensor([32.3323, 30.2924, 32.6657, 28.6776, 34.3523, 30.4057, 31.7975, 33.6143,
        30.5847, 30.2058], device='cuda:0', grad_fn=<NegBackward>)
191 100.0 tensor([32.3723, 30.2494, 32.6622, 28.9423, 34.5369, 30.0170, 31.9392, 33.7055,
        30.6962, 30.2786], device='cuda:0', grad_fn=<NegBackward>)
192 100.0 tensor([32.3469, 30.1906, 32.6127, 28.8937, 34.2288, 30.3905, 31.7079, 34.0351,
        30.7638, 30.3242], device='cuda:0', grad_fn=<NegBackward>)
193 100.0 tensor([32.5646, 30.0624, 32.3586, 28.8764, 34.0775, 30.5543, 31.9202, 34.1006,
        30.9363, 30.3223], device='cuda:0', grad_fn=<NegBackward>)
194 100.0 tensor([32.4644, 29.9947, 32.5419, 28.6222, 34.3